In [ ]:
from google.colab import auth, files
from google.cloud import bigquery
from datetime import datetime
import pytz

# Authenticate and set up the BigQuery client
auth.authenticate_user()
client = bigquery.Client(project='bq-rf5039')

def get_latest_table_name(client, dataset_id):
    query = f"""
        SELECT table_name
        FROM `{dataset_id}.INFORMATION_SCHEMA.TABLES`
        WHERE table_type = 'BASE TABLE'
        ORDER BY creation_time DESC
        LIMIT 1
    """
    query_job = client.query(query)
    results = query_job.result()
    for row in results:
        return row.table_name
    return None

# Get the latest tables
latest_melted_table = get_latest_table_name(client, 'bq-rf5039.MELTED')
latest_prices_table = get_latest_table_name(client, 'bq-rf5039.PRICES')

print(f"Latest table from MELTED: {latest_melted_table}")
print(f"Latest table from PRICES: {latest_prices_table}")

if latest_melted_table and latest_prices_table:
    # Generate a unique table name based on the current time
    paris_tz = pytz.timezone('Europe/Paris')
    current_time = datetime.now(paris_tz).strftime("%Y%m%d_%H%M%S")
    new_table_name = f'melted_lv_prices_{current_time}'

    # Create the SQL query
    query = f"""
        CREATE TABLE `bq-rf5039.MELTED_LV_PRICES.{new_table_name}` AS
        SELECT
            m.*,
            p.product_price,
            CASE
                WHEN p.product_price IS NOT NULL THEN m.volume * p.product_price
                ELSE NULL
            END AS product_sales
        FROM
            `bq-rf5039.MELTED.{latest_melted_table}` m
        LEFT JOIN (
            SELECT *
            FROM `bq-rf5039.PRICES.{latest_prices_table}`
            WHERE change_tracker != 'removed'
        ) p ON
            m.product_type = p.product_type
            AND m.year = p.year
            AND m.product_type IN ('starter', 'restarter', 'alternator')

    """

    # Execute the query
    query_job = client.query(query)
    query_job.result()  # Wait for the query to finish

    print(f"Table created: `bq-rf5039.MELTED_LV_PRICES.{new_table_name}`")
else:
    print("Latest tables not found in MELTED or PRICES datasets.")
